## VISAR etalon selection ##

For multi-channel VISAR systems, careful selection of the etalon thicknesses can improve the overall performance of the system while minimizing fringe jump ambiguity. Here, we'll consider a metric that we'll call the "velocity dynamic range" (VDR), which we'll define as the ratio of the mimimum velocity spacing between ambiguous solutions to the velocity uncertainty of the more-sensitive etalon.

In [2]:
# module imports and setup
from fractions import Fraction
import numpy as np
import bokeh.plotting as bkp
import bokeh.layouts as bkl
import bokeh.models as bkm
bkp.output_notebook()

Loading BokehJS ...

### 1-channel VISAR system
To start with, let's consider a 1-channel VISAR system with 5% fringe determination uncertainty.

In [3]:
vpf_A = 1  # velocity-per-fringe (fringe sensitivity) of channel A
df = 0.1  # assume 10% fringe determination uncertainty
dv_A = df * vpf_A # corresponding velocity uncertainty

# construct a toy visar velocity trace
t = np.linspace(0, 1, 101)
v = np.exp(-t)

fig = bkp.figure(x_axis_label='time', y_axis_label='velocity',
             plot_width=500, plot_height=300)
colors = ('blue', 'red')
for i in range(3):
    y = v + i * vpf_A
    fig.line(t, y, line_color=colors[0])
    for u in (dv_A, -dv_A):
        fig.line(t, y + u, line_color=colors[0], line_dash=(5,3))
bkp.show(fig)

In this case, the velocity can only be determined to the  $\pm$0.1 level (due to the 10% fringe determination uncertainty). In addition, there is a step ambiguity for velocity jumps greater than 1 (since vpf_A = 1). The VDR is then:

In [4]:
VDR = vpf_A / dv_A
print("{:.0f} = VDR for 1-channel VISAR system "
      "with {:.0f}% fringe uncertainty".format(VDR, df*100))

10 = VDR for 1-channel VISAR system with 10% fringe uncertainty


The actual velocity-per-fringe (vpf) of the etalon cancels out since the VDR is dimensionless. 

### 2-channel VISAR system
Let's now consider the addition of a second VISAR channel with a higher etalon sensistivity (thicker etalon, smaller vpf).

In [5]:
vpf_ratio = Fraction(2,3)
vpf_B = float(vpf_ratio) * vpf_A
dv_B = df * vpf_B

for i in range(vpf_ratio.denominator + 1):
    y = v + i * vpf_B
    fig.line(t, y, line_color=colors[1])
    for u in (dv_B, -dv_B):
        fig.line(t, y + u, line_color=colors[1], line_dash=(1,3))
bkp.show(fig)

Now, the velocity can be determined to $\pm$0.067 relying on the greater sensitivity of channel B. In addition, velocity jumps of about 1 can now be unambiguously rejected, since they would give an inconsistency between channel A and B that is greater than the sum of uncertainties in those channels. Instead, jumps need to be at least 2 before the next agreement between channels is found.

In [6]:
VDR = 2 * vpf_A / dv_B
print("{:.0f} = VDR for 2-channel VISAR system with {} vpf ratio "
      "and {:.0f}% fringe uncertainty".format(VDR, vpf_ratio, df*100))

30 = VDR for 2-channel VISAR system with 2/3 vpf ratio and 10% fringe uncertainty


So in this case we tripled the VDR with the addition of the second channel. But what is the best ratio?

### General 2-channel case ###

Let's first construct a function that computes the VDR of a 2-channel system with vpf ratio and fringe uncertainty as inputs. Here, we identify the channels to be in agreement if the velocities are closer than the sum of the channel velocity uncertainties. And we reduce the final uncertainty by the quadrature sum of the two channels.

In [7]:
def calc_VDR(ratio=2/3, uncertainty=0.1):
    """Return minimum number of phase jumps for given vpf ratio and fringe uncertainty."""
    ratio = float(ratio)
    jumps = np.arange(0, int(1/uncertainty) + 1) # span of possible phase jumps
    dv = np.mod(ratio * jumps + 0.5, 1) - 0.5 # velocity diff btwn channels at various jump conditions
    dvv = np.abs(np.subtract.outer(dv, dv)) # matrix of jump differences
    M = np.transpose((dvv + np.eye(len(jumps)) <= 2*uncertainty*(1 + ratio)).nonzero())
    return np.min(abs(M[:,0] - M[:,1])) / (uncertainty)# / np.hypot(1, ratio))

Now, let's calculate the VDR over a wide range of vpf ratios and fringe uncertainties:

In [8]:
Nr = 300 # number of ratio values
ustep = 0.001 # step between uncertainty values
ratios = np.linspace(0, 1, Nr+1)
ulim = 0.002, 0.17 # uncertainty range
uncertainties = np.arange(ulim[0], ulim[1]+1e-6, ustep)
VDR = np.empty((len(uncertainties), len(ratios)))
for i, u in enumerate(uncertainties):
    for j, r in enumerate(ratios):
        VDR[i,j] = calc_VDR(r, u)

Finally, let's construct an interactive view of the VDR.

In [50]:
# select reference ratio and reference uncertainty for lineouts
ref_ratio = 2/5
ref_uncertainty = 0.05

# figure settings
width, height = 300, 250
xlabel = "vpf ratio"
ylabel = "fringe uncertainty (%)"
zlabel = "VDR"
vdr_lim = (0, 200)

# image
p1 = bkp.figure(plot_width=width+50, plot_height=height,
                title=zlabel + " of 2-channel VISAR",
                x_range=(0,1), x_axis_label=xlabel,
                y_range=(0,100*ulim[1]), y_axis_label=ylabel)
color_mapper = bkm.LogColorMapper(palette="Viridis256", low=10, high=vdr_lim[1]+1)
p1.image([VDR], x=0, y=100*ulim[0], dw=1, dh=100*ulim[1], color_mapper=color_mapper)

ticker = bkm.FixedTicker(ticks=[10,20,50,100,200])
color_bar = bkm.ColorBar(color_mapper=color_mapper, ticker=ticker,
                         label_standoff=6, border_line_color=None, location=(0,0))
p1.add_layout(color_bar, 'right')

#lineout
lineout_color = 'red'
lineout_opts = dict(line_color=lineout_color, line_dash=(3,5))
p2 = bkp.figure(plot_width=width, plot_height=height,
                x_range=vdr_lim, x_axis_label=zlabel,
                y_range=p1.y_range, y_axis_label=ylabel)
p2.line(VDR[:,int(ref_ratio * Nr)], 100*uncertainties, line_color=lineout_color)
p1.line(ref_ratio*np.ones(2), (0, 100*ulim[1]), **lineout_opts)
p2.add_layout(bkm.Label(x=20,y=85*ulim[1], text="lineout, vpf ratio = {}".format(str(
              Fraction(ref_ratio).limit_denominator()))))

p3 = bkp.figure(plot_width=width+50, plot_height=height,
                x_range=p1.x_range, x_axis_label=xlabel,
                y_range=vdr_lim, y_axis_label=zlabel)
p3.line(ratios, VDR[int(ref_uncertainty/ustep),:], line_color=lineout_color)
p1.line(np.arange(2), 100*ref_uncertainty*np.ones(2), **lineout_opts)
p3.add_layout(bkm.Label(x=0.1,y=0.85*vdr_lim[1],
                        text="lineout, uncertainty = {:.0f}%".format(100*ref_uncertainty)))

# placeholder
p4 = bkp.figure(plot_width=width, plot_height=height,)
bkp.show(bkl.gridplot([p1, p2, p3, p4], ncols=2))

The funny triangular region is where there are additional benefits to resolving fringe ambiguities with a 2-channel system. Outside that region, there is just the ordinary multiple-measurements reduction in the velocity uncertainty. Etalon sensitivity ratios close to the ratios of whole numbers generally perform better, with the cutoff for higher denominators occuring at lower single-channel fringe determination uncertainties. For example, ratios of 1,2,3, or 4 to 5 work well for a 5% fringe uncertainty.

Lower etalon ratios tend to give slightly better VDR than their equivalents at higher numerators, but the advantage of the higher ratios is that if the low-sensitivity channel fails, you're less likely to incur a large penalty at resolving the fringe number with the remaining channel.